# Simple Scene Manipulation Workflow

**Zero training required!** Manipulate specific scenes in videos to increase/decrease attention-keyword alignment.

## Workflow
1. Load pre-trained model (InstructPix2Pix)
2. Specify video ID and scene index
3. Choose action: "increase" or "decrease"
4. Get edited video with that scene manipulated!

**That's it!** No training, no complex configuration.

## Step 1: Setup

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from GenAI_v3 import SceneManipulator

print("✓ Imports successful")

## Step 2: Initialize Manipulator

This loads the pre-trained model (will download on first run, ~5GB).

In [ ]:
# Initialize manipulator
manipulator = SceneManipulator(
    valid_scenes_file="../data/valid_scenes.csv",
    video_dir="../data/data_tiktok",  # Your videos are here: {video_id}.mp4
    output_dir="../outputs/genai_v3/manipulated_videos",
    method="instruct_pix2pix",  # Simpler, no masks needed
    device="cuda",  # Use "cpu" if no GPU
    use_scene_detection=False,  # False = faster (uses estimation)
)

print("\n✓ Manipulator initialized and ready!")

## Step 3: Manipulate a Specific Scene

**Example**: Increase attention-keyword alignment for scene 6 in video "123456"

In [ ]:
# Specify parameters
VIDEO_ID = "123456"  # ← Change this to your video ID
SCENE_INDEX = 6      # ← Change this to your scene number (1-indexed)
ACTION = "increase"  # ← "increase" or "decrease"

# Manipulate and replace scene in video
output_video = manipulator.manipulate_and_replace(
    video_id=VIDEO_ID,
    scene_index=SCENE_INDEX,
    action=ACTION,
    keyword=None,  # Auto-detect from valid_scenes.csv
    boost_strength=None,  # Auto-set (1.5 for increase, 0.5 for decrease)
    num_inference_steps=20,  # 10=fast, 20=balanced, 50=high quality
    blend_frames=5,  # Smooth blending at scene boundaries
)

print(f"\n✓ DONE! Output video: {output_video}")

## Optional: Preview Scene Location (Debugging)

If you want to verify which frames will be edited, use this preview function.

In [ ]:
# Preview where scene 6 is located in the video
manipulator.preview_scene_location(
    video_id=VIDEO_ID,
    scene_index=SCENE_INDEX,
)

# This will show:
# - Reference scene image
# - Video frames before, at start, middle, end, and after the scene

## Example 2: Decrease Alignment

In [ ]:
# Decrease attention for scene 3
output_video = manipulator.manipulate_and_replace(
    video_id="123456",
    scene_index=3,
    action="decrease",  # Make keyword LESS prominent
    num_inference_steps=20,
)

print(f"\n✓ Output: {output_video}")

## Example 3: Custom Boost Strength

In [ ]:
# Subtle increase (1.2× instead of default 1.5×)
output_video = manipulator.manipulate_and_replace(
    video_id="123456",
    scene_index=5,
    action="increase",
    boost_strength=1.2,  # Custom strength
    num_inference_steps=30,  # Higher quality
)

print(f"\n✓ Output: {output_video}")

## Example 4: Manipulate Multiple Scenes in One Video

In [ ]:
# Edit multiple scenes at once
output_video = manipulator.manipulate_multiple_scenes(
    video_id="123456",
    scene_actions={
        3: "increase",   # Boost scene 3
        6: "increase",   # Boost scene 6
        8: "decrease",   # Reduce scene 8
    },
    num_inference_steps=20,
)

print(f"\n✓ Output: {output_video}")

## Batch Processing: Multiple Videos

In [ ]:
import pandas as pd

# Load valid scenes
scenes_df = pd.read_csv("../data/valid_scenes.csv")

# Get unique video IDs
video_ids = scenes_df['video_id'].unique()[:5]  # First 5 videos

print(f"Processing {len(video_ids)} videos...\n")

# For each video, boost scene 6 (if exists)
for video_id in video_ids:
    # Check if scene 6 exists
    if 6 in scenes_df[scenes_df['video_id'] == video_id]['scene_number'].values:
        try:
            output_video = manipulator.manipulate_and_replace(
                video_id=str(video_id),
                scene_index=6,
                action="increase",
                num_inference_steps=15,  # Faster for batch
            )
            print(f"✓ {video_id} -> {output_video}\n")
        except Exception as e:
            print(f"✗ {video_id} failed: {e}\n")
    else:
        print(f"⊘ {video_id} - Scene 6 does not exist\n")

print("\n✓ Batch processing complete!")

## Summary

### What We Did
1. ✅ Loaded pre-trained model (no training!)
2. ✅ Specified video ID and scene index
3. ✅ Chose action: "increase" or "decrease"
4. ✅ Got edited video with manipulated scene

### Performance
- **Speed**: ~20-30 seconds per scene
- **Quality**: Natural-looking edits
- **Training**: Zero!

### Parameters to Adjust
- `boost_strength`: 1.2 (subtle) → 2.0 (strong)
- `num_inference_steps`: 10 (fast) → 50 (high quality)
- `blend_frames`: 3 (abrupt) → 10 (very smooth)
- `use_scene_detection`: False (fast) → True (accurate)

### Next Steps
1. Deploy edited videos for A/B testing
2. Collect engagement metrics (CTR, CVR, watch time)
3. Compare performance between original and edited videos
4. Iterate on boost strength based on results